In [1]:
import numpy as np
import pandas as pd

In [44]:
fbi = pd.read_excel('..\..\Downloads/table_8_offenses_known_to_law_enforcement_new_york_by_city_2013.xls')

In [45]:
# info about data
fbi.iloc[0:3, 0]

0                             NEW YORK
1    Offenses Known to Law Enforcement
2                        by City, 2013
Name: Table 8, dtype: object

In [46]:
# dropping first 3 rows
df = fbi.drop([0, 1, 2], axis=0)

In [47]:
# using row as column titles
df.rename(columns=df.iloc[0], inplace=True)

In [48]:
df = df.reindex(df.index.drop(3)).reset_index(drop=True)

In [49]:
df

,City,Population,Violent crime,Murder and nonnegligent manslaughter,Rape (revised definition)1,Rape (legacy definition)2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson3
0,Adams Village,1861,0,0,NaN,0,0,0,12,2,10,0,0
1,Addison Town and Village,2577,3,0,NaN,0,0,3,24,3,20,1,0
2,Akron Village,2846,3,0,NaN,0,0,3,16,1,15,0,0
3,Albany,97956,791,8,NaN,30,227,526,4090,705,3243,142,NaN
4,Albion Village,6388,23,0,NaN,3,4,16,223,53,165,5,NaN
5,Alfred Village,4089,5,0,NaN,0,3,2,46,10,36,0,NaN
6,Allegany Village,1781,3,0,NaN,0,0,3,10,0,10,0,0
7,Amherst Town,118296,107,1,NaN,7,31,68,2118,204,1882,32,3
8,Amityville Village,9519,9,0,NaN,2,4,3,210,16,188,6,1
9,Amsterdam,18182,30,0,NaN,0,12,18,405,99,291,15,0


In [50]:
# notes for columns (Rape and Arson)
note = df.iloc[348:, 0]

In [51]:
# dropping notes from df
df.drop([348, 349, 350], inplace=True)

In [52]:
df.iloc[:, 4].count()

0

In [53]:
# dropping 'revised Rape column', contains only NaN
df.drop('Rape\n(revised\ndefinition)1', axis=1, inplace=True)

In [54]:
df.rename(columns={'Violent\ncrime': 'violent_crime', 'Murder and\nnonnegligent\nmanslaughter':'murder', 'Rape\n(legacy\ndefinition)2':'rape', 'Aggravated\nassault': 'aggrvt_asslt', 'Property\ncrime':'property_crime', 'Larceny-\ntheft': 'larceny_theft', 'Motor\nvehicle\ntheft':'motor_veh_theft', 'Robbery': 'robbery', 'Arson3': 'arson', 'Burglary':'burglary'}, inplace=True)

In [55]:
df['feat_murder'] = df['murder'].apply(lambda x: 1 if x>0 else 0)

In [56]:
df['feat_robbery'] = df['robbery'].apply(lambda x: 1 if x>0 else 0 )

In [57]:
df['population_sqr'] = np.square(df['Population'])

In [58]:
# arson data not shown if full calendar year is not given
# assume 0
df.isnull().sum()

City                 0
Population           0
violent_crime        0
murder               0
rape                 0
robbery              0
aggrvt_asslt         0
property_crime       0
burglary             0
larceny_theft        0
motor_veh_theft      0
arson              161
feat_murder          0
feat_robbery         0
population_sqr       0
dtype: int64

In [59]:
df['arson'] = df['arson'].fillna(0)

In [60]:
df.rename(index=df.iloc[:, 0], inplace=True)

In [61]:
df.drop('City', axis=1, inplace=True)

In [62]:
from sklearn.linear_model import LinearRegression

In [63]:
a = df[['Population', 'population_sqr', 'feat_murder', 'feat_robbery']]

In [64]:
b = df[['Population', 'population_sqr', 'feat_murder', 'feat_robbery']].values

In [65]:
regr = LinearRegression()

In [66]:
regr.fit(b, df.property_crime)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [67]:
regr.coef_

array([ 3.46570268e-02, -2.11108019e-09,  1.51866535e+01, -9.62774363e+01])

In [68]:
regr.intercept_

-109.57533562257424

In [69]:
regr.score(b, df.property_crime)

0.9961247104988709

In [70]:
import statsmodels.formula.api as smf
from statsmodels.sandbox.regression.predstd import wls_prediction_std


In [86]:
# Write out the model formula.
# Your dependent variable on the right, independent variables on the left
# Use a ~ to represent an '=' from the functional form
linear_formula = 'property_crime ~ Population+population_sqr+feat_murder+feat_robbery'

# Fit the model to our data using the formula.
lm = smf.ols(formula=linear_formula, data=df).fit()

In [87]:
lm.params

,0,1,2,3,4,5,6,7,8,9,...,229,230,231,232,233,234,235,236,237,238
Intercept,5.421011e-17,4.423545e-17,-1.079432e-15,3.989864e-17,1.0,-1.214306e-17,3.512815e-17,1.314053e-16,8.630249e-17,1.244664e-16,...,2.168404e-17,1.301043e-17,-3.903128e-17,1.604619e-17,1.994932e-17,-7.632783e-17,1.994932e-17,1.951564e-17,-3.903128e-17,1.301043e-17
Population[T.615],5.000000e-01,4.900594e-17,2.224783e-16,-2.038300e-17,-0.5,-5.702903e-17,2.818926e-17,-2.103352e-17,3.491131e-17,-4.076600e-17,...,-2.775558e-17,-3.361027e-17,-4.770490e-17,5.637851e-17,-6.570265e-17,5.507747e-17,-5.377643e-17,-1.006140e-16,-1.012645e-16,-4.683753e-17
Population[T.656],-5.551115e-17,-1.951564e-17,-8.673617e-19,-2.775558e-17,-0.5,-1.023487e-16,1.582935e-16,5.000000e-01,3.903128e-18,-1.019150e-16,...,-5.204170e-18,0.000000e+00,-3.469447e-17,1.058181e-16,-2.602085e-17,9.540979e-17,0.000000e+00,0.000000e+00,-1.561251e-17,-5.204170e-17
Population[T.658],-1.291827e-16,1.658829e-17,5.000000e-01,-3.387048e-16,-0.5,-1.051676e-17,-2.341877e-17,-8.104411e-16,-1.857306e-16,-6.288373e-17,...,-3.035766e-17,-2.032879e-17,2.081668e-17,-2.862294e-17,-1.387779e-17,-2.949030e-16,-3.534499e-17,-3.629367e-17,1.387779e-16,-2.775558e-17
Population[T.762],-7.762888e-17,-6.591949e-17,5.000000e-01,2.077331e-16,-0.5,5.009014e-17,1.322727e-16,-6.591949e-17,8.673617e-17,2.480655e-16,...,-8.890458e-18,2.905662e-17,-8.326673e-17,-5.290907e-17,1.214306e-17,1.726050e-16,-1.181780e-17,1.301043e-17,5.898060e-17,-3.469447e-18
Population[T.829],-1.387779e-17,6.071532e-18,-9.454243e-17,-1.387779e-17,-0.5,-1.565588e-16,-8.933826e-17,1.361758e-16,1.485357e-17,-5.182486e-17,...,6.245005e-17,3.046608e-17,3.816392e-17,-3.469447e-18,3.642919e-17,1.162265e-16,6.418477e-17,-3.295975e-17,-2.775558e-17,5.377643e-17
Population[T.872],5.000000e-01,1.272853e-16,2.255141e-17,2.602085e-17,-0.5,2.450297e-17,1.557185e-16,-1.500536e-16,-2.233456e-17,1.639314e-16,...,6.938894e-18,-1.908196e-17,4.727121e-17,-2.775558e-17,7.285839e-17,9.194034e-17,3.295975e-17,3.469447e-18,6.245005e-17,-2.602085e-17
Population[T.977],1.669671e-16,1.283695e-16,1.561251e-17,4.320546e-17,-0.5,-1.092876e-16,9.584347e-17,-3.604972e-17,-1.734723e-17,-2.524023e-16,...,4.293441e-17,-5.898060e-17,1.734723e-18,-6.938894e-18,0.000000e+00,7.632783e-17,-5.724587e-17,5.421011e-17,5.984796e-17,4.336809e-17
Population[T.980],-4.943962e-17,-1.192622e-18,1.775923e-16,5.000000e-01,-0.5,-5.231275e-17,1.769418e-16,1.949396e-16,1.491862e-16,7.155734e-18,...,5.898060e-17,-3.469447e-18,1.257675e-16,1.387779e-17,1.214306e-17,1.804112e-16,-1.561251e-17,-3.469447e-17,-2.168404e-18,4.531965e-17
Population[T.997],-5.876376e-17,7.632783e-17,1.615461e-17,5.778798e-17,-0.5,5.000000e-01,-5.009014e-17,9.974660e-18,-1.561251e-17,2.428613e-17,...,-7.112366e-17,-3.469447e-18,-2.862294e-17,-7.112366e-17,-1.301043e-16,8.543513e-17,0.000000e+00,-1.734723e-17,-3.838076e-17,3.209238e-17


In [85]:
lm.pvalues

ValueError: shapes (348,239) and (348,239) not aligned: 239 (dim 1) != 348 (dim 0)

In [72]:
df

,Population,violent_crime,murder,rape,robbery,aggrvt_asslt,property_crime,burglary,larceny_theft,motor_veh_theft,arson,feat_murder,feat_robbery,population_sqr
Adams Village,1861,0,0,0,0,0,12,2,10,0,0,0,0,3463321
Addison Town and Village,2577,3,0,0,0,3,24,3,20,1,0,0,0,6640929
Akron Village,2846,3,0,0,0,3,16,1,15,0,0,0,0,8099716
Albany,97956,791,8,30,227,526,4090,705,3243,142,0,1,1,9595377936
Albion Village,6388,23,0,3,4,16,223,53,165,5,0,0,1,40806544
Alfred Village,4089,5,0,0,3,2,46,10,36,0,0,0,1,16719921
Allegany Village,1781,3,0,0,0,3,10,0,10,0,0,0,0,3171961
Amherst Town,118296,107,1,7,31,68,2118,204,1882,32,3,1,1,13993943616
Amityville Village,9519,9,0,2,4,3,210,16,188,6,1,0,1,90611361
Amsterdam,18182,30,0,0,12,18,405,99,291,15,0,0,1,330585124
